In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from numpy.core.fromnumeric import argsort, mean, squeeze
from torch import tensor
from torch.functional import norm
import matplotlib.pyplot as plt
import numpy as np
from torch import optim
import torch.nn.functional as F
import math as m
import time
import os 
import random
from PIL import Image
from torch.autograd import Variable
from PIL import Image
import numpy
import tensorflow as tf
from pathlib import Path
import pickle
import numpy as np
import torch
import torchvision
import torch.nn.functional as F

from google.colab import drive
drive.mount('/gdrive') 

if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("GPU")
else:
  device = torch.device("cpu")
  print("CPU")




class NLR3(nn.Module):
    def __init__(self,netin,netout,nethidden):
      super().__init__()
      self.netmodel= torch.nn.Sequential(torch.nn.Linear(netin, nethidden),torch.nn.Tanh(),torch.nn.Linear(nethidden, netout))
    def myforward (self,inv):
      outv=self.netmodel(inv)
      return outv


# with open('/gdrive/My Drive/dataset/Features172QueryStructureallF.txt', 'rb') as fp:
#        AllData=pickle.load( fp)


with open('/gdrive/My Drive/dataset/netcout_18_2804jn.txt', 'rb') as fp:
       inp=pickle.load(fp)
with open('/gdrive/My Drive/dataset/target_18_2804jn.txt', 'rb') as fp:
       target=pickle.load(fp)

target=torch.tensor(target).to(device)
inp=torch.tensor(inp).to(device)

hidden=2500
l_r=0.071
epoch=100000
save_duration=5
min_error=0.1
batch_size=25691
print_duration=2
print(inp.shape)
print(target.shape)
total_size=target.shape[0]
model_mlp=NLR3(inp.shape[1],target.shape[1],hidden).to(device)
if(os.path.isfile('/gdrive/My Drive/dataset/UltraNetB18_CO2804.pth')):    
  print('Load Saved Model From Google Drive')
  model_mlp.load_state_dict(torch.load( '/gdrive/My Drive/dataset/UltraNetB18_CO2804.pth', map_location=torch.device(device) ))
else:
  print('Load old Model From Google Drive')
  model_mlp.load_state_dict(torch.load( '/gdrive/My Drive/dataset/net152models/UltraNetB152_2500.pth', map_location=torch.device(device) ))
loss_fn=torch.nn.CosineSimilarity()

prev_loss=  torch.mean((1-loss_fn(target,model_mlp.myforward(inp))))
print('prevloss',prev_loss)
optimizer=torch.optim.SGD(model_mlp.parameters(), lr=l_r)
optimizer.zero_grad()

for j in range(epoch):
    tot_loss=0
    for i in range(int(total_size/batch_size)):
      netout=model_mlp.myforward(inp[i*(batch_size):(i+1)*batch_size]).to(device)
      target_batch=target[batch_size*i:batch_size*(i+1)].to(device)
      loss = torch.mean((1-loss_fn(target_batch,netout)))
      tot_loss +=loss
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    if (j%print_duration==0):
      print('iteration:',j, 'total loss',tot_loss,'avg loss', loss,tot_loss/(total_size/batch_size))
    if(j%save_duration==0):
      torch.save(model_mlp.state_dict(), '/gdrive/My Drive/dataset/UltraNetB18_CO2804.pth')
      if ((tot_loss/(total_size/batch_size))<prev_loss) :
         print('enhanced saved  ')
         prev_loss=tot_loss/(total_size/batch_size)
         torch.save(model_mlp.state_dict(), '/gdrive/My Drive/dataset/UltraNetB18_CO2804_best.pth')


print('Finished Training')
torch.save(model_mlp.state_dict(), '/gdrive/My Drive/dataset/final_UltraNetB18_CO2804.pth')  
  

Mounted at /gdrive
GPU
torch.Size([205528, 512])
torch.Size([205528, 512])
Load Saved Model From Google Drive
prevloss tensor(0.1443, device='cuda:0', grad_fn=<MeanBackward0>)
iteration: 0 total loss tensor(1.1560, device='cuda:0', grad_fn=<AddBackward0>) avg loss tensor(0.1327, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1445, device='cuda:0', grad_fn=<DivBackward0>)
iteration: 2 total loss tensor(1.1560, device='cuda:0', grad_fn=<AddBackward0>) avg loss tensor(0.1327, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1445, device='cuda:0', grad_fn=<DivBackward0>)
iteration: 4 total loss tensor(1.1561, device='cuda:0', grad_fn=<AddBackward0>) avg loss tensor(0.1328, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1445, device='cuda:0', grad_fn=<DivBackward0>)
iteration: 6 total loss tensor(1.1562, device='cuda:0', grad_fn=<AddBackward0>) avg loss tensor(0.1329, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.1445, device='cuda:0', grad_fn=<DivBackward0>)
iteration: 8 tot